### 8.4 평행 이동
영상의 원점을 기준으로 모든 화소를 동일하게 가로, 세로 방향으로 이동

CV에서는 y값을 증가시키면(더하면) 이미지가 아래로 이동함

In [1]:
# 8.4.1 영상 평행 이동
import numpy as np, cv2

# 좌표가 인수범위 내에 있는지 확인
def contain(p, shape):
    return 0 <= p[0] < shape[0] and 0 <= p[1] < shape[1] # 범위 이내이면 True, 아니면 False

def translate(img, pt):
    dst = np.zeros(image.shape, img.dtype)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            x, y = np.subtract((j, i), pt)
            if contain((y, x), img.shape): # 범위를 벗어나면 굳이 계산하여 삽입하지 않음
                dst[i,j] = img[y, x]
    return dst

image = cv2.imread("images_08/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")


dst1 = translate(image, (30, 80))
dst2 = translate(image, (-70, -50))

cv2.imshow("image", image)
cv2.imshow("dst1 (30, 80)", dst1)
cv2.imshow("dst2 (-70, -50)", dst2)
cv2.waitKey(0)

### 8.5 회전

역방향 회전은

x = xcosT + ysinT

y = -xsinT + ycosT

--------

영상 회전은 좌측최상단을 기준으로 돌아가지만 일반적으로 중심값을 기준으로 돌리는 회전을 사용하므로,

영상의 가운데를 좌측최상단으로 이동하여 회전 후 다시 원래 자리로 이동함


In [12]:
# 8.5.1 영상 회전
import numpy as np, cv2, math
from Common.interpolation import bilinear_value
from Common.functions import contain

# 원점 기준 회전 변환
def rotate(img, degree):
    dst = np.zeros(img.shape[:2], img.dtype)
    radian = (degree/180) * np.pi # 회전 변환 각도를 라디안으로 변형
    sin, cos = np.sin(radian), np.cos(radian)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            y = -j*sin + i*cos
            x = j*cos + i*sin
            if contain((y, x), img.shape): # 범위를 벗어난 위치는 연산하지 않음
                dst[i, j] = bilinear_value(img, [x, y])
    return dst

# 특정 중심을 가지고 움직임
def rotate_pt(img, degree, pt):
    dst = np.zeros(img.shape[:2], img.dtype)
    radian = (degree/180) * np.pi
    sin, cos = math.sin(radian), math.cos(radian)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            jj, ii = np.subtract((j, i), pt) # 중심좌표로 평행 이동

            y = -jj*sin + ii*cos
            x = jj*cos + ii*sin
            x, y = np.add((x,y), pt)
            if contain((y, x), img.shape):
                dst[i, j] = bilinear_value(img, [x, y])
    return dst

image = cv2.imread("images_08/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

center = np.divmod(image.shape[::-1], 2)[0]
print(center)
dst1 = rotate(image, 20)
dst2 = rotate_pt(image, 20, center)

cv2.imshow("image", image)
cv2.imshow("dst1 : rotated on (0,0)", dst1)
cv2.imshow("dst2 : rotated on center", dst2)
cv2.waitKey(0)

[113 113]


-1

In [1]:
# 8.5.2 마우스로 영상 회전
import numpy as np, cv2
from Common.interpolation import rotate_pt

def calc_angle(pts):
    d1 = np.subtract(pts[1], pts[0]).astype(float)
    d2 = np.subtract(pts[2], pts[0]).astype(float)
    angle1 = cv2.fastAtan2(d1[1], d1[0])
    angle2 = cv2.fastAtan2(d2[1], d2[0])
    return (angle2 - angle1)

def draw_point(x, y):
    pts.append([x, y])
    print("좌표 : ", len(pts), [x, y])
    cv2.circle(tmp, (x, y), 2, 255, 2)
    cv2.imshow("image", tmp)

def onMouse(event, x, y, flags, param):
    global tmp, pts
    if (event == cv2.EVENT_LBUTTONUP and len(pts) == 0): draw_point(x, y)
    if (event == cv2.EVENT_LBUTTONDOWN and len(pts) == 1): draw_point(x, y)
    if (event == cv2.EVENT_LBUTTONUP and len(pts) == 2): draw_point(x, y)

    if len(pts) == 3:
        angle = calc_angle(pts)
        print("회전각: %3.2f" % angle)
        dst = rotate_pt(image, angle, pts[0])
        cv2.imshow("image", dst)
        tmp = np.copy(image)
        pts = []
    
image = cv2.imread("images_08/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")
tmp = np.copy(image)
pts = []

cv2.imshow("image", image)
cv2.setMouseCallback("image", onMouse, 0)
cv2.waitKey(0)


좌표 :  1 [104, 97]
좌표 :  2 [142, 130]
좌표 :  3 [142, 130]
회전각: 0.00
좌표 :  1 [95, 117]
좌표 :  2 [79, 81]
좌표 :  3 [79, 81]
회전각: 0.00
좌표 :  1 [66, 60]
좌표 :  2 [141, 105]
좌표 :  3 [96, 148]
회전각: 40.21


-1